In [ ]:
# cell for importing packages for the first time
# import Pkg; Pkg.add("StaticArrays")
# import Pkg; Pkg.add("Revise")


import Pkg; Pkg.add("@time")

In [ ]:
ENV["PYTHON"]=""
import Pkg;
Pkg.add("PyCall")

In [ ]:
Pkg.add("PyPlot")

In [2]:
# using Plots

using StaticArrays
import LinearAlgebra

push!(LOAD_PATH, pwd())

module PhysConst
    export CL,Gr,QE,MSOL,MSUN,ARAD,SGMB,RGAS,
            PC,RSUN,RSUN,YR,MSOLYR,GAM53,KPE,PARSEC,KPE,KPD,M_MW
    CL = 2.997925E10;
    Gr = 6.67384E-8; 
    QE=4.80325E-10;
    MSOL = 1.989E33;
    MSUN = 1.989E33;

    ARAD = 7.56464E-15; 
    SGMB = ARAD*CL/4

    RGAS = 8.31E7; 
    PC = 3.085678E18;
    MP = 1.672661E-24; 
    M_U = 1.660531E-24
    RSOL = 6.95E10
    RSUN = 6.95E10
    YR = 365*24*3600.
    MSOLYR = 1.989e33/YR

    GAM = 5/3.

    KPE = 0.4
    PARSEC=3.0E18
    KPD = 10.

    M_MW=1.

    tst=1

end



Main.PhysConst

In [ ]:
using PyPlot
# use x = linspace(0,2*pi,1000) in Julia 0.6
x = range(0; stop=2*pi, length=1000); y = sin.(3 * x + 4 * cos.(2 * x));
plot(x, y, color="red", linewidth=2.0, linestyle="--")
title("A sinusoidally modulated sinusoid")

In [3]:

#-------------------------------------- STEP 1 -------------------------------------- #
# define necessary data structures

# module SpaceRadGrid


using StaticArrays
import Base.@kwdef

    const IX = 1
    const IY = 2
    const IZ = 3

    const BIGNUMBER = 10.0^50



    @kwdef struct SGridPar
        

        Nactive = 20;
    
        nghost = 1 #N of ghost cells at every boundary.
            
        Nx = Nactive;   # N of active cells
        Ny = Nactive;
        Nz = Nactive;
        
        Nxtot = Nx +  2*nghost;
        Nytot = Ny +  2*nghost;
        Nztot = Nz +  2*nghost;
    
        is = 1 + nghost; 
        ie = Nxtot -nghost;        

        js = 1 + nghost;  
        je = Nytot - nghost;
        
        ks = 1 + nghost;
        ke = Nztot - nghost;
            
        x1s = -1.
        x1e = 1.
    
        x2s = -1.
        x2e = 1.

        x3s = -1.
        x3e = 1.
        
    end

    @kwdef struct RGridPar
    
        Nfreq = 1 ; # ν - N bins 
        NumPhi = 5
        NumTheta = 5
    
        Nang = NumPhi*NumTheta
    
        N_fre_ang = Nfreq*Nang    
    
    end


# ------ define instances ----------

sg= SGridPar()

@show rad = RGridPar()

# ----------------------------------

x1 = Array{Float64}(undef, sg.Nxtot)
x2 = Array{Float64}(undef, sg.Nytot)
x3 = Array{Float64}(undef, sg.Nztot)

x1b = Array{Float64}(undef, sg.Nxtot)
x2b = Array{Float64}(undef, sg.Nytot)
x3b = Array{Float64}(undef, sg.Nztot)


@time ir = zeros(Float64, rad.N_fre_ang, sg.Nx, sg.Ny, sg.Nz); # radiation intensity



function MakeGrid(x1, x2, x3, x1b, x2b, x3b, sg)  #staggered grid

    # add check sizes
    
    xtmp = range(sg.x1s, sg.x1e; length = sg.Nxtot - 2sg.nghost)
    
#     @show sg.is, sg.ie sg.Nx sg.nghost size(xtmp) size(x1) size(sg.is:sg.ie) size(2:2)
    
    x1[sg.is:sg.ie] .= xtmp[:] 
    x1[sg.is-1] = 2*x1[sg.is]- x1[sg.is+1]
    x1[sg.ie+1] = 2*x1[sg.ie] -x1[sg.ie-1]

    xtmp = range(sg.x2s, sg.x2e; length = sg.Nytot - 2sg.nghost)    

    x2[sg.js:sg.je] .= xtmp[:]     
    x2[sg.js-1] = 2*x2[sg.js]- x2[sg.js+1]
    x2[sg.je+1] = 2*x2[sg.je] -x2[sg.je-1]

    xtmp = range(sg.x3s, sg.x3e; length = sg.Nztot - 2sg.nghost)    

    x3[sg.ks:sg.ke] .= xtmp[:]     
    x3[sg.ks-1] = 2*x3[sg.ks]- x3[sg.ks+1]
    x3[sg.ke+1] = 2*x3[sg.ke] -x3[sg.ke-1]
    
    
    x1b[1] = x1[1] - 0.5(x1[sg.is]-x1[1])    
    for i in sg.is:(sg.ie+1)
        x1b[i] = x1[i-1] + 0.5(x1[i]-x1[i-1])
    end

    x2b[1] = x2[1] - 0.5(x2[sg.js]-x2[1])    
    for j in sg.js:(sg.je+1)
        x2b[j] = x2[j-1] + 0.5(x2[j]-x2[j-1])
    end

    x3b[1] = x3[1] - 0.5(x3[sg.ks]-x3[1])    
    for k in sg.ks:(sg.ke+1)
        x3b[k] = x3[k-1] + 0.5(x3[k]-x3[k-1])
    end
    
    
    
end


MakeGrid(x1,x2,x3,x1b,x2b,x3b,sg)
@show x3
@show x3b

@kwdef mutable struct LongRay1
#     first time calculation of the long ray
    
    numOfElm::Int32
    
    dir = Vector{Float64}  #direction
    
    ijkOfCellCrossed:: Matrix{Int32}
        
    xyzPos ::Matrix{Float64}

end


#  allocate one big array
@time begin
    
    NumElemInRay = 2*max(sg.Nxtot,sg.Nytot,sg.Nztot)


    rayOne = LongRay1(NumElemInRay, zeros(Float64,3), Array{Int32}(undef,3,NumElemInRay),  Array{Float64}(undef,3,NumElemInRay) )
    

    
    arrOfLongRay = Array{LongRay1, 4}(undef, (rad.N_fre_ang, sg.Nxtot, sg.Nytot, sg.Nztot));    

    for m = 1:rad.N_fre_ang, i=1:sg.Nxtot, j=1:sg.Nytot, k=1:sg.Nztot
        
        arrOfLongRay[m,i,j,k] = LongRay1(NumElemInRay, zeros(Float64,3), Array{Int32}(undef,3,NumElemInRay), Array{Float64}(undef,3,NumElemInRay) )

    end

    
end


# end



rad = RGridPar() = RGridPar(1, 5, 5, 25, 25)
  0.013935 seconds (28.90 k allocations: 3.062 MiB)
x3 = [-1.1052631578947367, -1.0, -0.8947368421052632, -0.7894736842105263, -0.6842105263157895, -0.5789473684210527, -0.47368421052631576, -0.3684210526315789, -0.2631578947368421, -0.15789473684210525, -0.05263157894736842, 0.05263157894736842, 0.15789473684210525, 0.2631578947368421, 0.3684210526315789, 0.47368421052631576, 0.5789473684210527, 0.6842105263157895, 0.7894736842105263, 0.8947368421052632, 1.0, 1.1052631578947367]
x3b = [-1.157894736842105, -1.0526315789473684, -0.9473684210526316, -0.8421052631578947, -0.736842105263158, -0.631578947368421, -0.5263157894736842, -0.42105263157894735, -0.3157894736842105, -0.21052631578947367, -0.10526315789473684, 0.0, 0.10526315789473684, 0.21052631578947367, 0.3157894736842105, 0.42105263157894735, 0.5263157894736842, 0.631578947368421, 0.736842105263158, 0.8421052631578947, 0.9473684210526316, 1.0526315789473684]
  0.483830 seconds (1.89 M

In [4]:
# module RadiationTransfer

x3c = Array{Float64}(undef, 3);
xn = Array{Float64}(undef, 3)

function FirstTimeTraceGridOverOneLongRay(rayOne::LongRay1, sg::SGridPar)


#     @show rayOne.dir
    
    dist = 0.0   
    distToNextCell .= BIGNUMBER

    xn .= x3c

    itermax = 2 * sg.Nx

    for iter in 1:itermax #max length, may be smaller than actual

#     @show iter



        for (n_it, norm_i ) in enumerate(rayOne.dir) #QUESTION: maybe iter over pre-calclulated 1/norms


          xc = x3c[n_it] 

             if iter > 1 

                itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, rayOne.dir[n_it])

                @debug  " debug: (itmp, iter)= " , itmp, iter

                xn[n_it] = xbi_zip[n_it][itmp]                                            


             elseif iter==1 #first step from the middle of the cell

                if norm_i>0                
                   itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, rayOne.dir[n_it])                     
                elseif norm_i <0               
                    itmp = ijk_tmp[n_it] = ijk_pos[n_it]  #belongs to the same index                                
                else                
                    itmp = ijk_pos[n_it]            
                end

                xn[n_it] = xbi_zip[n_it][itmp]

             end


             if norm_i != 0                      
                distToNextCell[n_it] = abs((xn[n_it] - xc)/rayOne.dir[n_it])
             else
                distToNextCell[n_it]=BIGNUMBER
             end


        #println("iter= ",iter," xn=", xn, " ===> ", "distToNextCell[",n_it,"]=",distToNextCell[n_it])



        end #for loop over norm_i     

            posIndxToUpdate = argmin(distToNextCell)
            dist = distToNextCell[posIndxToUpdate]


        ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one
    

        #println("ijk_pos[posIndxToUpdate = ", posIndxToUpdate, "]",  "  =  ",   ijk_pos  )


        xn .= x3c + rayOne.dir*dist
        x3c .= xn

        #update rayOne
        rayOne.numOfElm = iter
        rayOne.ijkOfCellCrossed[:,iter] .= ijk_pos
        rayOne.xyzPos[:,iter] .= xn

#         println("xn=", xn)

        if ijk_pos[posIndxToUpdate] == ijk_max[posIndxToUpdate] || ijk_pos[posIndxToUpdate] == ijk_min[posIndxToUpdate] || ijk_pos[posIndxToUpdate] == 1 || iter > 100 
            break; 
        end      

    end #over ray

end # FirstTimeTraceGridCalcOneLongRay


function InitRayDirs(rad::RGridPar, sg::SGridPar)
    th = range(0, pi; length = rad.NumPhi)
    ph = range(0, 2pi-pi/rad.NumTheta; length = rad.NumTheta)
@show th, ph
    norm = zeros(Float64, rad.Nang, 3);
    

    m=1
    for (i,ph_i) in enumerate(ph)
        for (j,th_j) in enumerate(th)
                        
            norm[m, IX] = sin(th_j)sin(ph_i)
            norm[m, IY] = sin(th_j)cos(ph_i)                 
            norm[m, IZ] = cos(th_j)
                        
        m+=1;            
        end
    end

    # LinearAlgebra.norm(norm[9,:])

    return(norm)
end



function InitArrOfLongRay(norm::Array{Float64,2}, rayArr::Array{LongRay1,4}, rad::RGridPar, sg::SGridPar)
 
    for m in 1:rad.N_fre_ang, ipos=1:sg.Nxtot, jpos=1:sg.Nytot, kpos=1:sg.Nztot
        rayArr[m, ipos,jpos,kpos].dir[:]  =  norm[m,:]
    end
           
end

norm = InitRayDirs(rad,sg)

InitArrOfLongRay(norm, arrOfLongRay, rad, sg);
   
# checkup
let 
    m = rand(1:rad.N_fre_ang)
    arrOfLongRay[m, rand(1:sg.Nx),rand(1:sg.Ny),rand(1:sg.Nz) ].dir - norm[m,:]    
end
   
function PlotRay(ray, sg)
    
    x = ray.xyzPos[1,1:ray.numOfElm]
    y = ray.xyzPos[2,1:ray.numOfElm]   
    z = ray.xyzPos[3,1:ray.numOfElm]      
    
    @show ray.numOfElm,x,y,z

    plot3d(x,y,z,
        xlabel = "x", ylabel = "y", zlabel = "z", 
        xlim= (sg.x1s,sg.x1e),ylim= (sg.x2s,sg.x2e),zlim= (sg.x3s,sg.x3e))    
end




distToNextCell = fill(BIGNUMBER,3) #used in FirstTimeTraceGridOverOneLongRay()

ijk_pos = Array{Int64}(undef,3)
ijk_max = [ sg.ie+1, sg.je+1, sg.ke+1 ] #max ijk index 
ijk_min = [ sg.is, sg.js, sg.ks ] #min index
ijk_tmp = [0, 0, 0]

xbi_zip = [x1b, x2b, x3b]

ijk_pos_s = [rand(sg.is:sg.ie),  rand(sg.js:sg.je),  rand(sg.ks:sg.ke) ]
ijk_pos_s = [argmin(abs.(x1.-0.)), argmin(abs.(x2.-0.)), argmin(abs.(x3.-0.))];

for m in 1:rad.N_fre_ang #, ipos=1:sg.Nxtot, jpos=1:sg.Nytot, kpos=1:sg.Nztot
    
    ijk_pos .= ijk_pos_s
    
    x3c .= [ x1[ijk_pos[IX]], x2[ijk_pos[IY]], x3[ijk_pos[IZ]] ]
    @show m , x3c , ijk_pos

#     if m > 
#         break        
#     end
        
    rayOne = arrOfLongRay[m, ijk_pos[IX], ijk_pos[IY], ijk_pos[IZ]]
    
    
    FirstTimeTraceGridOverOneLongRay(rayOne, sg)    
    
 
    #     plt=PlotRay(rayOne, sg); display(plt)    
    
end





# display(plt)    
# inline()
# end #module RadiationTransfer





(th, ph) = (0.0:0.7853981633974483:3.141592653589793, 0.0:1.413716694115407:5.654866776461628)
(m, x3c, ijk_pos) = (1, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (2, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (3, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (4, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (5, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (6, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (7, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (8, [-0.05263157894736842, -0.05263157894736842, -0.05263157894736842], [11, 11, 11])
(m, x3c, ijk_pos) = (9, [-0.05263157894736842, -0.0526315

In [ ]:
ray = arrOfLongRay[rand(1:rad.Nang), ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]
    
@show ijk_pos_s ray.dir

plt=plot3d(ray.xyzPos[1,1:ray.numOfElm],
        ray.xyzPos[2,1:ray.numOfElm],
        ray.xyzPos[3,1:ray.numOfElm],
        xlabel = "x", ylabel = "y", zlabel = "z", 
        xlim= (sg.x1s,sg.x1e),ylim= (sg.x2s,sg.x2e),zlim= (sg.x3s,sg.x3e))

 
display(plt)


In [ ]:
# function xyzToijk(xin::Float64, yin::Float64, zin::Float64, x1::Array{Float64,1}, x2::Array{Float64,1},x3::Array{Float64,1})
#     i = argmin( abs.(x1 .-xin )); 
#     j = argmin(abs.(x2.-yin)); 
#     k = argmin(abs.(x3.-zin))    
#     return(i,j,k)
# end

# typeof(x1)

    
i = argmin(abs.(x1.-0.)); 


# GetIndxGiveCoords1(0., 0., 0., x1, x2, x3)




<h1>
    <b>
   Schwarzschild-Schuster 2BV problem:
    </b>
    
</h1>

<!-- <font size="4"> -->



$$   
 \begin{equation}    
    p\frac{d^2u}{dt^2} = u - S\mbox{,}
 \end{equation}
$$  
where $t=\tau$, $p=\mu^2 = const$, $S=const$
    
Boundary conditions:
    
 $$
 \text{at}\quad t = \tau_1\simeq 0: \quad \alpha \, u'(t) + \beta \, u(t) = \gamma \mbox{,} 
 $$
    
 $$
 \text{at}\quad t = \tau_N: \quad \alpha_1 \, u'(t) + \beta_1 \, u(t) = \gamma_1 \mbox{.} 
 $$
Source function in SS-approximation:
\begin{equation}
    S(t) = \frac{1}{2}\left( I^{+}(t) + I^{-}(t) \right)\equiv u(t)
\end{equation}


<!-- </font> -->


In [4]:
function DensMatrixSolvAlongTheRay(xGrid, rayOne::LongRay1, sg::SGridPar, rad::RGridPar)
    
    
    Nn = rad.Nfreq; # ν - N bins 
    Nd = rayOne.numOfElm; # x - bins 
    Nmat = Nd*Nn
    
    Mat = zeros(Float32, Nmat, Nmat)
    Rhs = zeros(Float32, Nmat) #right side    
    
    alph = 1.
    bet = -1.
    gam = 0

    alph1 = 1.
    
    bet1 = 1.
    gam1 = 1.

    Sd = 2.
    Sbet =1.
    bm = 1.
    
    p=1.


    for i = 1:Nmat
        for i = 1:Nmat

            iSubMatRaw = div(i + Nn-1, Nn)         

            if iSubMatRaw == 1 
                Rhs[i] = -gam                   
            elseif iSubMatRaw == Nd 
                Rhs[i] = gam1
            else
                Rhs[i] = Sd+Sbet
            end


            for j = 1:Nmat


            jSubMatCol = div(j + Nn-1, Nn)

            iloc = i - Nn * (iSubMatRaw-1)        
            jloc = j - Nn * (jSubMatCol-1)

            if iSubMatRaw == 1 #left BC    


                dx_i = xGrid[2]-xGrid[1]
                dx_im05 = dx_i 
                dx_ip05 = dx_i        
                fPar = [dx_im05, dx_i, dx_ip05, 1]

                if jSubMatCol == 1
                    B1 = -bet + alph/dx_ip05

                    Mat[i,j] = iloc==jloc ? B1 : 0 #only diag    

                elseif jSubMatCol == 2

                    C1 = -alph/dx_ip05

                    Mat[i,j] = iloc==jloc ? C1 : 0

                end                        

            elseif iSubMatRaw > 1 && iSubMatRaw < Nd
    #           the main body of the matrix

                dx_i = xGrid[iSubMatRaw+1]-xGrid[iSubMatRaw]
                dx_im05 = dx_i 
                dx_ip05 = dx_i

                    if iSubMatRaw == jSubMatCol  && iloc==jloc
#                         B = 1 + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i
                        
                        B = 1 - bm + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i
                        
                        Mat[i,j] = B
                    elseif iSubMatRaw == jSubMatCol+1 && iloc==jloc                     
                        A =  -p /(dx_i*dx_im05)                     
                        Mat[i,j] = A                    
                    elseif iSubMatRaw == jSubMatCol-1                    
                        C=-(p/(dx_i*dx_ip05))                                        
                        Mat[i,j] = C
                    else                                        
                        Mat[i,j] = 0                                                        
                    end


            elseif iSubMatRaw == Nd #right BC                
                dx_i = xGrid[Nd]-xGrid[Nd-1]
                dx_im05 = dx_i 
                dx_ip05 = dx_i

                if jSubMatCol == Nd-1

                    AN = -(alph1/dx_im05)

                    Mat[i,j] = iloc==jloc ? AN : 0

                elseif jSubMatCol == Nd

                    BN = bet1 + alph1/dx_im05

                    Mat[i,j] = iloc==jloc ? BN : 0

                end

            end

            end


        end
    end

# end

    display(Mat)
    # display(Rhs)
    # @show Rhs
    Mat_inv = inv(Mat)
    # Mat_inv*Mat

    ytestNumeric1 = Mat_inv*Rhs

    
    
    end  #RTFotrieAlongTheRay

    
function SchwarzSchustRT()

        ray = arrOfLongRay[1, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]
    
        @show  ray.dir
                    
        tauGrid = range(0., 1; length = ray.numOfElm)    
        #typeof(tauGrid)
    
    
        DensMatrixSolvAlongTheRay(tauGrid, ray, sg, rad)
    

    
end



SchwarzSchustRT()


11×11 Array{Float32,2}:
   11.0   -10.0     0.0     0.0     0.0  …     0.0     0.0     0.0     0.0
 -100.0   200.0  -100.0     0.0     0.0        0.0     0.0     0.0     0.0
    0.0  -100.0   200.0  -100.0     0.0        0.0     0.0     0.0     0.0
    0.0     0.0  -100.0   200.0  -100.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0  -100.0   200.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0  -100.0  …     0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0     -100.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0      200.0  -100.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0     -100.0   200.0  -100.0     0.0
    0.0     0.0     0.0     0.0     0.0        0.0  -100.0   200.0  -100.0
    0.0     0.0     0.0     0.0     0.0  …     0.0     0.0   -10.0    11.0

ray.dir = [0.0, 0.0, 1.0]


11-element Array{Float32,1}:
 1.6833389
 1.8516695
 1.9900022
 2.0983353
 2.17667
 2.2250032
 2.2433367
 2.2316697
 2.1900027
 2.1183357
 2.0166688

<h1>
<b>
Rybicki & Hummer (91) method
</b>
</h1>

$$   
 \begin{equation}    
    p\frac{d^2u}{dt^2} = u - S\mbox{,}
 \end{equation}
$$  


<b> Matrix  </b> eq: 
$$
-A_i u_{i-1} + B_i u_{i} -C_i u_{i+1} = S_i
$$

Coefficients have a slightly different form  better numerical derivative representation on staggered grid to allow for implicit contribution from the $b_m u_m$ from $S$. 

In [29]:
function ProgonRybHumAlongTheRay(ray::LongRay1, sg::SGridPar, rad::RGridPar)
    Nmat = ray.numOfElm
    
    u = zeros(Float64, Nmat)
    
    A = zeros(Float64, Nmat)
    B = zeros(Float64, Nmat)    
    C = zeros(Float64, Nmat)
    D = zeros(Float64, Nmat)
    Z = zeros(Float64, Nmat)
    S = zeros(Float64, Nmat) #Rhs

    p=1
    alph = 1.
    bet = -1.
    gam = 0
    alph1 = 1.
    
    bet1 = 1.
    gam1 = 1.
    
    Sd = 2.
    Sbet =1.
    bm = 0
    
    dx_i = LinearAlgebra.norm(ray.xyzPos[:,2].-ray.xyzPos[:,1]) 
    dx_ip05 = dx_i
    
    A[1] = 0.
    B[1] = -bet + alph/dx_ip05
    C[1] = alph/dx_ip05
    S[1] = -gam
    D[1] = C[1]/B[1]
    Z[1] = S[1]/B[1]
    
    C[Nmat]=0.
    
    for i = 2 : Nmat-1 #forward                        

        dx_i = LinearAlgebra.norm(ray.xyzPos[:,i+1].-ray.xyzPos[:,i])                  
        dx_im05 = dx_i         
        dx_ip05 = dx_i  
        S[i] = Sd+Sbet
        
        A[i] = p /(dx_i*dx_im05)         
        B[i] = 1. - bm + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i        
        C[i] = p/(dx_i*dx_ip05)  
                        
        k1 = 1/(B[i] - A[i]D[i-1])        
        D[i] = k1*C[i]        
        Z[i] = k1*( S[i]  + A[i]*Z[i-1])                     
    end 

    dx_im05 = dx_i         
    dx_ip05 = dx_i 
    S[Nmat] = gam1
    
    C[Nmat]=0.
    D[Nmat]=0.

    A[Nmat] = (alph1/dx_im05)
    B[Nmat] = bet1 + alph1/dx_im05
    
    k1 = 1/(B[Nmat] - A[Nmat]*D[Nmat-1])
    
    Z[Nmat] = k1*( S[Nmat]  + A[Nmat]*Z[Nmat-1])                     
    u[Nmat] = Z[Nmat]
    
    for i = Nmat-1:-1:1  #backsweep

        u[i] = D[i]u[i+1]+Z[i]
        
    end #for                    
    
    return(u)
end #RTAlongTheRay

function ProgonRybHumAlongTheRayAppndxA(ray::LongRay1, sg::SGridPar, rad::RGridPar)
    
    Nmat = ray.numOfElm
    
    u = zeros(Float64, Nmat)
        
    D = zeros(Float64, Nmat)
    H = zeros(Float64, Nmat)
    F = zeros(Float64, Nmat)
    Z = zeros(Float64, Nmat)    

    p=1
    alph = 1.
    bet = -1.
    gam = 0
    alph1 = 1.
    
    bet1 = 1.
    gam1 = 1.
    
    Sd = 2.
    Sbet =1.
    bm = 0
    
    dx_i = LinearAlgebra.norm(ray.xyzPos[:,2].-ray.xyzPos[:,1]) 
    dx_ip05 = dx_i
    
    
    B1 = -bet + alph/dx_ip05
    C1 = alph/dx_ip05
    S1 = -gam

    H[1] = B1-C1    
    F[1] = H[1]/C1
    Z[1] = S1/B1    
    
    
    for i = 2 : Nmat-1 #forward                        

        dx_i = LinearAlgebra.norm(ray.xyzPos[:,i+1].-ray.xyzPos[:,i])                  
        dx_im05 = dx_i         
        dx_ip05 = dx_i  
        Si = Sd+Sbet
        
        Ai = p /(dx_i*dx_im05)         
        Bi = 1 - bm + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i        
        Ci = p/(dx_i*dx_ip05)  

        H[i] = -Ai + Bi - Ci                
        F[i]=(H[i]+(Ai*F[i-1])/(1+F[i-1]))/Ci        
        Z[i]=(Si+Ai*Z[i-1])/(1+F[i])/Ci      
        
    end 

    dx_im05 = dx_i         
    dx_ip05 = dx_i 
    SN = gam1
    
    CN=0.
    AN = (alph1/dx_im05)
    BN = bet1 + alph1/dx_im05
    
    Dnm1 = 1/(1+F[Nmat-1])    
    Z[Nmat] = (SN+AN*Z[Nmat-1])/( BN - AN*Dnm1)                         
    u[Nmat] = Z[Nmat]

    for i = Nmat-1:-1:1  #backsweep

        u[i] = u[i+1]/(1+F[i]) + Z[i]
        
    end #for                    
    
    return(u)
    
end #RTAlongTheRay



ray = arrOfLongRay[1, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]

u1 = ProgonRybHumAlongTheRay(ray, sg, rad)


u.=0
u=ProgonRybHumAlongTheRayAppndxA(ray, sg, rad);

println( "Δu/u=", LinearAlgebra.norm((u-u1)./u1) )



Δu/u=6.424123762405305e-15


No documentation found.

`LinearAlgebra.norm1` is a `Function`.

```
# 2 methods for generic function "norm1":
[1] norm1(x::Union{DenseArray{T,1}, Base.ReinterpretArray{T,1,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, Base.ReshapedArray{T,1,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, SubArray{T,1,A,I,L} where L where I<:Tuple{Vararg{Union{Int64, AbstractRange{Int64}, Base.AbstractCartesianIndex},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, DenseArray}, Array{T,N} where N}) where T<:Union{Float32, Float64} in LinearAlgebra at /Users/avdorodn/Applications/Julia-1.4.app/Contents/Resources/julia/share/julia/stdlib/v1.4/LinearAlgebra/src/dense.jl:102
[2] norm1(x) in LinearAlgebra at /Users/avdorodn/Applications/Julia-1.4.app/Contents/Resources/julia/share/julia/stdlib/v1.4/LinearAlgebra/src/generic.jl:537
```


In [ ]:

# module RadTransfer
    

ijk_pos = Array{Int64}(undef,3)
ijk_max = [ sg.ie+1, sg.je+1, sg.ke+1 ] #max ijk index 
ijk_min = [ sg.is, sg.js, sg.ks ] #min index

distToNextCell = fill(BIGNUMBER,3) #used in FirstTimeTraceGridOverOneLongRay()


norm[1, IX, :, :, :] .=  1
norm[1, IY, :, :, :] .= -1
norm[1, IZ, :, :, :] .= -1


iOf_0 = argmin(abs.(x1)); jOf_0 = argmin(abs.(x2)); kOf_0 = argmin(abs.(x3))
# ijk_pos .= [sg.is, sg.js, sg.ks]
# ijk_pos .= [iOf_0, div(sg.je,2), div(sg.ke,2)]
# ijk_pos .= [sg.ie, sg.je, sg.ke]

ijk_pos .= [iOf_0,  jOf_0,  kOf_0]
ijk_tmp = [0, 0, 0]



rayOne.dir .= [norm[1, IX, 1, 1, 1], norm[1, IY, 1, 1, 1] , norm[1, IZ, 1, 1, 1]]


rayOne.dir=LinearAlgebra.normalize(rayOne.dir)

@show rayOne.dir

# rayOne.rayOne.dir = norm3 #update ray direction
# norm3 = [norm[1, IX, 1, 1, 1], norm[1, IY, 1, 1, 1] , norm[1, IZ, 1, 1, 1]]
# should replace 1/norm just in case when Float norm==0.
#          one2norm_i = max(abs())

xbi_zip = [x1b, x2b, x3b]



@show ijk_pos

x3c = [ x1[ijk_pos[IX]], x2[ijk_pos[IY]], x3[ijk_pos[IZ]] ]

# @show x3c
# @show x1
# @show x1b

# @show x2
# @show x2b

# @show x3
# @show x3b

# @show xbi_zip[2]


FirstTimeTraceGridOverOneLongRay(rayOne, sg)







# end #module RadTransfer

In [ ]:
#-------------------------------------- STEP 2 -------------------------------------- #



# @show size(norm);



In [ ]:
# logger=Logging.SimpleLogger(stderr,Logging.Debug)

function TestOneRay(ray, sg)
    xyz_s=Vector{Float64}(undef,3)    
    xyz_e=Vector{Float64}(undef,3)    
    xyz_0e=Vector{Float64}(undef,3)
    
    xyz_s .= ray.xyzPos[:,1]
    
    xyz_e = ray.xyzPos[:,ray.numOfElm]

     dist =LinearAlgebra.norm( xyz_e[:].-xyz_s[:]) 
    
    xyz_0e = xyz_s + ray.dir*dist
    
    
    @show xyz_e, xyz_0e, dist
    
    println("|| rend - rs || = ",  LinearAlgebra.norm( xyz_e[:].-xyz_0e[:]) )
    
end


TestOneRay(rayOne, sg)



In [ ]:
module debugByPlot

using Plots
# x= LinRange(0,30,10)

# import 

x = [0.5, 1.]
y = [0., 1.]
z = [0., 0.1];


plt=plot3d(x,y,z)



end


In [ ]:
# function mtest()
# L = M = N = Int(5e2);

# vec_vec_vec = Array{Array{Vector}}(L);

# for i = 1:L vec_vec_vec[i] = [zeros(N) for j = 1:M] end;

# vec_mat = Array{Matrix}(L);

# fill!(vec_mat, rand(M, N));

# arr = rand(L, M, N);

# @time  for i = 1:L for j = 1:M for k = 1:N vec_vec_vec[i][j][k] += 1; end; end; end; 

# @time  for i = 1:L for j = 1:M for k = 1:N vec_mat[i][j,k] += 1; end; end; end; 

# @time  for k = 1:N for j = 1:M for i = 1:L arr[i,j,k] += 1; end; end; end; 

# end
# mtest()



# mutable struct SegmentData{
#  /*! cell data along the trajectory */
#   /*!  current index of cell along the ray, phi index is the same */
#   /*! if the source is symmetrical */
#   int i,k;    
#   float dl; //length element    
# }SegmentData;

# RayCellInd1 = Array{Int32}(undef,1)
# RayCellInd2 = Array{Int32}(undef,1)
# RayCellInd3 = Array{Int32}(undef,1)
# push!(RayCellInd1, 1)
# RayCellInd1 .=0
# RayCellInd2 .=0
# RayCellInd3 .=0


# create array of ray directions, mu, for every ijk cell (nx,ny,nz) components
# for every ray; 
# nang - number of rays per cell 



# RayData = Array of ( (iCross, jCross, kCross), NcrossCells,(istart, jstart, kstart)=3)
# NcrossCells=3
# @time RayData = zeros(3,  zeros(3),  3)





In [ ]:
copysign(1, 0)

In [ ]:
import Pkg; Pkg.add("Plotly")

In [ ]:
# using Plotly
# default(show = true)
# plotly()
